In [4]:
from minio import Minio
from minio.error import S3Error

client = Minio(
    "localhost:9000",  # Replace with your MinIO server address
    access_key="ml_testing",
    secret_key="ml_testing",
    secure=False  # Set to True if using HTTPS
)

# Create a bucket
bucket_name = "ml-bucket"
try:
    client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except S3Error as err:
    print(f"Error occurred: {err}")

Bucket 'ml-bucket' created successfully.


In [24]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as pltpoetry 


import random
import sys
from dvclive import Live
from sklearn.metrics import r2_score

In [7]:
df = pd.read_pickle('data/ev_pareto_on_gen.pkl')

# Split the data into features and target
X = df[["rotorAngle","rotorDiameter","Bias"]]
y = df[['LapTime','Durability','MotorPower','Torque']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Initialize the CatBoostRegressor
model = CatBoostRegressor(
    iterations=2000, learning_rate=0.05, depth=4, loss_function='MultiRMSE', verbose=200)

# Fit the model
model.fit(X_train, y_train)




with Live(save_dvc_exp=True) as live:
    live.log_param("learning_rate", 0.05)
    for depth in range(2,5):

        live.log_param("depth", depth)

        # Initialize the CatBoostRegressor
        model = CatBoostRegressor(
            iterations=2000, learning_rate=0.05, depth=depth, loss_function='MultiRMSE', verbose=200)

        # Fit the model
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_test)
        column_names = ['LapTime','Durability','MotorPower','Torque']
        predictions = model.predict(X_test)
        y_test_pred = pd.DataFrame(predictions,columns=column_names)

        live.log_metric("test/r2", r2_score(y_test['Torque'], y_test_pred['Torque']), plot=False)

        live.next_step()